# asdf

최규빈  
2023-09-25

# asdf

## C. `s.apply(변환함수)`

`-` 예시: 원소별로 처음3개의 숫자만 출력

In [39]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DV2022/master/posts/FIFA23_official_data.csv')
s = df.Height

## D. `s.str`

`-` 예시1: 원소별로 처음 3개의 숫자만 출력

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DV2022/master/posts/FIFA23_official_data.csv')
s = df.Height

`-` 예시2: 원소별로 isupper를 수행

In [18]:
s = pd.Series(['A','B','C','d','e','F'])
s

## E. `s.astype()`

`-` 예시1: 원소의 타입을 모두 `int`형으로 변경

In [27]:
s = pd.Series(list('12345'))
s

`-` 예시2: 원소별로 (1) 처음3개의 문자열만 취하고 (2) 원소의 타입을 모두
인트형으로 변경

In [601]:
df = pd.read_csv('https://raw.githubusercontent.com/guebin/DV2022/master/posts/FIFA23_official_data.csv')
s = df.Height